In [1]:
from google.colab import drive
import sys
drive.mount('/content/drive')
project_dir = '/content/drive/MyDrive/Colab Notebooks/project/final/'
sys.path.append(project_dir+'services')

Mounted at /content/drive


In [2]:
%%writefile "/content/drive/MyDrive/Colab Notebooks/project/final/services/recommendation_service.py"
"""
Recommendation service for the music recommender app.
Provides music recommendations based on user preferences and behavior.
"""
import os
import pandas as pd
import random

class RecommendationService:
    """Service for providing music recommendations."""

    def __init__(self, songs_csv_path, history_csv_path, users_csv_path):
        """
        Initialize the recommendation service.

        Args:
            songs_csv_path (str): Path to the songs CSV file
            history_csv_path (str): Path to the listening history CSV file
            users_csv_path (str): Path to the users CSV file
        """
        self.songs_csv_path = songs_csv_path
        self.history_csv_path = history_csv_path
        self.users_csv_path = users_csv_path
        self.load_data()

    def load_data(self):
        """Load necessary data from CSV files."""
        # Load songs data
        if os.path.exists(self.songs_csv_path):
            self.songs_df = pd.read_csv(self.songs_csv_path)
        else:
            self.songs_df = pd.DataFrame(columns=['song_id', 'artist', 'song', 'genre'])

        # Load listening history data
        if os.path.exists(self.history_csv_path):
            self.history_df = pd.read_csv(self.history_csv_path)
        else:
            self.history_df = pd.DataFrame(columns=[
                'history_id', 'user_id', 'song_id', 'emotion_before',
                'emotion_after', 'timestamp', 'is_recommended'
            ])

        # Load users data
        if os.path.exists(self.users_csv_path):
            self.users_df = pd.read_csv(self.users_csv_path)
        else:
            self.users_df = pd.DataFrame(columns=[
                'user_id', 'username', 'favorite_artists', 'recommended_artists'
            ])

    def recommend_songs(self, user_id, count=5, mood='sad'):
        """
        Recommend songs for a user based on favorites and listening history.

        Args:
            user_id (int): The user ID
            count (int): Number of songs to recommend
            mood (str): Current mood of the user

        Returns:
            pandas.DataFrame: Recommended songs
        """
        if self.songs_df.empty:
            return pd.DataFrame()

        # Get user's favorite artists
        user_row = self.users_df[self.users_df['user_id'] == user_id]
        if user_row.empty:
            return pd.DataFrame()

        fav_artists_str = user_row.iloc[0].get('favorite_artists', '')
        if pd.isna(fav_artists_str) or not fav_artists_str:
            # If no favorite artists, recommend random songs
            return self.songs_df.sample(min(count, len(self.songs_df)))

        favorite_artists = [a.strip() for a in fav_artists_str.split(',') if a.strip()]

        # Get songs by favorite artists
        favorite_songs = self.songs_df[self.songs_df['artist'].isin(favorite_artists)]

        if favorite_songs.empty:
            # If no songs by favorite artists, recommend random songs
            return self.songs_df.sample(min(count, len(self.songs_df)))

        # Get user's listening history
        if not self.history_df.empty:
            user_history = self.history_df[self.history_df['user_id'] == user_id]
            played_song_ids = user_history['song_id'].unique().tolist() if not user_history.empty else []

            # Filter out already played songs
            if played_song_ids:
                favorite_songs = favorite_songs[~favorite_songs['song_id'].isin(played_song_ids)]

        # Find mood-improving songs
        mood_improving_songs = self._get_mood_improving_songs(mood)
        if not mood_improving_songs.empty and len(mood_improving_songs) >= count:
            # If we have enough mood-improving songs, use those
            return mood_improving_songs.sample(min(count, len(mood_improving_songs)))

        # If we don't have enough favorite songs, add some random ones
        if len(favorite_songs) < count:
            # Select all available favorites
            recommendations = favorite_songs.copy()

            # Add random songs that aren't by favorite artists
            remaining_count = count - len(recommendations)
            if remaining_count > 0:
                other_songs = self.songs_df[~self.songs_df['artist'].isin(favorite_artists)]
                if not other_songs.empty:
                    # Add random songs from other artists
                    random_songs = other_songs.sample(min(remaining_count, len(other_songs)))
                    recommendations = pd.concat([recommendations, random_songs], ignore_index=True)
        else:
            # Select random subset of favorite songs
            recommendations = favorite_songs.sample(count)

        return recommendations

    def _get_mood_improving_songs(self, mood):
        """
        Get songs that have improved mood in the past.

        Args:
            mood (str): Current mood

        Returns:
            pandas.DataFrame: Songs that improved mood
        """
        if self.history_df.empty:
            return pd.DataFrame()

        # Find records where user mood improved
        if mood == 'sad':
            # Look for sad to happy/excited/relaxed transitions
            mood_improved = self.history_df[
                (self.history_df['emotion_before'] == 'sad') &
                (self.history_df['emotion_after'].isin(['happy', 'excited', 'relaxed']))
            ]
        else:
            # For other moods, just find songs that led to happy or excited
            mood_improved = self.history_df[
                self.history_df['emotion_after'].isin(['happy', 'excited'])
            ]

        if mood_improved.empty:
            return pd.DataFrame()

        # Get the song IDs that improved mood
        mood_song_ids = mood_improved['song_id'].unique().tolist()

        # Get the corresponding songs
        return self.songs_df[self.songs_df['song_id'].isin(mood_song_ids)]

    def get_song_by_id(self, song_id):
        """
        Get a song by ID.

        Args:
            song_id (int): The song ID

        Returns:
            pandas.Series or None: The song data if found, otherwise None
        """
        song_rows = self.songs_df[self.songs_df['song_id'] == song_id]

        if song_rows.empty:
            return None

        return song_rows.iloc[0]

    def get_random_song(self):
        """
        Get a random song.

        Returns:
            pandas.Series or None: A random song or None if no songs available
        """
        if self.songs_df.empty:
            return None

        return self.songs_df.sample(1).iloc[0]

    def get_user_recommendations(self, user_id):
        """
        Get stored recommendations for a user.

        Args:
            user_id (int): The user ID

        Returns:
            list: List of recommended artist names
        """
        user_row = self.users_df[self.users_df['user_id'] == user_id]

        if user_row.empty:
            return []

        rec_str = user_row.iloc[0].get('recommended_artists', '')

        if pd.isna(rec_str) or not rec_str:
            return []

        return [a.strip() for a in rec_str.split(',') if a.strip()]

Overwriting /content/drive/MyDrive/Colab Notebooks/project/final/services/recommendation_service.py
